# Frequent Sequence Mining

## Algorithm
### Suffix Array

In [69]:
test_string = 'banana\nanaK'
test_strings = test_string.split('\n')
print(test_strings)

['banana', 'anaK']


In [20]:
# suffix array
# https://louisabraham.github.io/notebooks/suffix_arrays.html

def suffix_array_oneliner(s):
    return [(suffix, rank) for suffix, rank in sorted((s[i:], i) for i in range(len(s)))]

def suffix_array_wo_sort(s):
    return [suffix for suffix in (s[i:] for i in range(len(s)))]

suffix_array_wo_sort(test_string)

['banana\nanaK',
 'anana\nanaK',
 'nana\nanaK',
 'ana\nanaK',
 'na\nanaK',
 'a\nanaK',
 '\nanaK',
 'anaK',
 'naK',
 'aK',
 'K']

### Frequent n-grams

k-common substrings problem (generalisation of Longest_common_substring)  
https://en.wikipedia.org/wiki/Longest_common_substring_problem

Find the most frequent (longest) substring O(n log n): suffix array -> longest common prefix (LCP) array (**suffix tree**)
https://mediatum.ub.tum.de/doc/1094574/1094574.pdf  
https://cs.stackexchange.com/questions/9555/computing-the-longest-common-substring-of-two-strings-using-suffix-arrays
http://www.roman10.net/2012/03/16/suffix-array-part-3-longest-common-substring-lcs/

In [85]:
# https://stackoverflow.com/questions/40556491/how-to-find-the-longest-common-substring-of-multiple-strings
from functools import partial, reduce
from itertools import chain
from typing import Iterator

from collections import Counter


def ngram(seq: str, n: int) -> Iterator[str]:
    return (seq[i: i+n] for i in range(0, len(seq)-n+1))


def allngram(seq: str, minn=1, maxn=None) -> Iterator[str]:
    lengths = range(minn, maxn) if maxn else range(minn, len(seq))
    ngrams = map(partial(ngram, seq), lengths)
    return set(chain.from_iterable(ngrams))


def frequent_ngrams(strings, min_support=None, topn=5):
    # 1.split traces into ngrams
    seqs_ngrams = map(allngram, strings)

    # 2.count ngram frequencies
    counts = Counter(chain.from_iterable(seqs_ngrams))

    # 3.filter frequent substrings
    # set frequency threshold if not specified
    if not min_support:
        most_frequent_s = [s for s, count in counts.most_common(topn)]
        # maximum frequency
#         most_frequent1 = counts.most_common(1)[0]
#         min_support = most_frequent1[1]
    else:
#         print(min_support)
        most_frequent={string: count for string, count in counts.items() if count >= min_support}
    #     print(most_frequent)
        most_frequent_s = list(most_frequent.keys())

    # 4.drop substrings
    most_frequent_s.sort(key=len, reverse=True)
#     print(most_frequent_s)
    lfss = [most_frequent_s[0]]
    for s in most_frequent_s[1:]:
        overlap = False
        for lfs in lfss:
            if s in lfs:
                overlap = True
                break
        if not overlap:
            lfss.append(s)
    # result: longest frequent substrings with counts
    return lfss, [counts[s] for s in lfss]


print(frequent_ngrams(test_strings))

(['ana'], [2])


## Mine Conversation Log

In [73]:
# get conversation log
import requests
data_path = "https://raw.githubusercontent.com/svakulenk0/conversation_mining/master/data/"
log_name = "1_dstc1.stringenc.txt"
log_string = requests.get(data_path+log_name).text
# print(len(log_string))
# experiment on a log snippet
log_snip = log_string[:200]
print(log_snip)

traces = log_string.split('\n')
print('\n')
print("%d traces"%len(traces))

DFAFEFCFEEFCGGEEFEFCGFEFCGFEGGE
DFEFEFEFCFC
DFFEGFEGEFCGFEGGGFAFEEEFFAFEEE
DFFEFCFEFEEFCGGFEFCGFEFEFCGFEGFEGGGFE
DFFEFCGFEEFCFCGFEGGGFA
DFEFAFEFCFEFCGFEGFEGFEGGGFEFEFFEFCGFEGFEFCGFE
DFFEFCFEFCFEEEFCFC


15867 traces


In [88]:
# dictionary: https://github.com/svakulenk0/conversation_mining/blob/master/data/1_dstc1.stringenc.dict.txt
print(frequent_ngrams(traces, topn=5))
print(frequent_ngrams(traces, min_support=10000))  # TODO min support as a fraction of the number of traces
# TODO plot frequency distribution

(['DF', 'FE'], [15831, 15468])
(['FEFC', 'EFCG', 'FCGF', 'GFE', 'DFF', 'FFE', 'FA'], [11446, 10195, 10985, 10725, 12471, 13602, 10428])


In [ ]:
# TODO look up sample traces